# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
#import nltk
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('omw-1.4')
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
#from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier

from sklearn.metrics import classification_report, confusion_matrix

from sklearn.model_selection import GridSearchCV

#from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

import pickle

In [3]:
# load data from database
engine = create_engine('sqlite:///disasterresponse.db')
df = pd.read_sql('messages', con = engine)

X = df['message'] 
Y = df.iloc[:, 4:]

In [4]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [5]:
def old_tokenize(text):
    #case normalization and punctuation removal:
    text = text.lower().strip()
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)

    #tokenization
    words = word_tokenize(text)

    #stop word removal
    words = [w for w in words if w not in stopwords.words("english")]

    #stemming
    words = [PorterStemmer().stem(w) for w in words]

    #lemmatization
    words = [WordNetLemmatizer().lemmatize(w) for w in words]
    
    return words

In [6]:
def tokenize(text):
    #punctuation removal:
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)

    #tokenization
    tokens = word_tokenize(text)

    #stop word removal
    tokens = [t for t in tokens if t not in stopwords.words("english")]

    #stemming and lemmatization
    stem = PorterStemmer()
    lem = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens: 
        clean_tok = stem.stem(tok)
        clean_tok = lem.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)   
   
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [8]:
vect = CountVectorizer(tokenizer = tokenize)
tfidf = TfidfTransformer(smooth_idf = False) #optional

##classifier
dtree_clf = MultiOutputClassifier(DecisionTreeClassifier(), n_jobs = -1)

In [9]:
#transform training data
X_train_counts = vect.fit_transform(X_train)
X_train_tfidf = tfidf.fit_transform(X_train_counts)

#train the classifier
dtree_clf.fit(X_train_tfidf, y_train)

MultiOutputClassifier(estimator=DecisionTreeClassifier(), n_jobs=-1)

In [10]:
#transform test data
X_test_counts = vect.transform(X_test)
X_test_tfidf = tfidf.transform(X_test_counts)

#predict test
y_pred = dtree_clf.predict(X_test_tfidf)

In [11]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('dtree_clf', MultiOutputClassifier(DecisionTreeClassifier(), n_jobs = -1)),
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 21)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
category_names = Y.columns

In [14]:
for i in range(len(Y.columns)):
        print(Y.columns[i])
        print(classification_report(y_test.iloc[:, i], y_pred[:, i]))

related
              precision    recall  f1-score   support

           0       0.52      0.51      0.52      1897
           1       0.84      0.85      0.85      5912

    accuracy                           0.77      7809
   macro avg       0.68      0.68      0.68      7809
weighted avg       0.77      0.77      0.77      7809

request
              precision    recall  f1-score   support

           0       0.91      0.92      0.91      6476
           1       0.58      0.55      0.57      1333

    accuracy                           0.85      7809
   macro avg       0.74      0.74      0.74      7809
weighted avg       0.85      0.85      0.85      7809

offer
              precision    recall  f1-score   support

           0       1.00      0.99      1.00      7779
           1       0.00      0.00      0.00        30

    accuracy                           0.99      7809
   macro avg       0.50      0.50      0.50      7809
weighted avg       0.99      0.99      0.99      780

In [15]:
accuracy = (y_pred == y_test).mean()
print(accuracy)

related                   0.768728
request                   0.854783
offer                     0.990908
aid_related               0.698809
medical_help              0.908055
medical_products          0.938532
search_and_rescue         0.961327
security                  0.967730
military                  0.961327
child_alone               1.000000
water                     0.955692
food                      0.937380
shelter                   0.932898
clothing                  0.987322
money                     0.966961
missing_people            0.984121
refugees                  0.956332
death                     0.960558
other_aid                 0.824561
infrastructure_related    0.901780
transport                 0.939045
buildings                 0.948649
electricity               0.978358
tools                     0.989499
hospitals                 0.983737
shops                     0.992829
aid_centers               0.981048
other_infrastructure      0.932642
weather_related     

In [16]:
print(classification_report(y_test, y_pred, target_names = category_names, zero_division = 0))

                        precision    recall  f1-score   support

               related       0.84      0.85      0.85      5912
               request       0.58      0.55      0.57      1333
                 offer       0.00      0.00      0.00        30
           aid_related       0.63      0.64      0.64      3227
          medical_help       0.41      0.36      0.38       617
      medical_products       0.38      0.33      0.35       399
     search_and_rescue       0.30      0.26      0.28       223
              security       0.10      0.12      0.11       129
              military       0.42      0.27      0.33       275
           child_alone       0.00      0.00      0.00         0
                 water       0.66      0.63      0.65       503
                  food       0.71      0.72      0.72       860
               shelter       0.61      0.62      0.61       678
              clothing       0.56      0.52      0.54       111
                 money       0.24      

### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000020D70896310>)),
  ('tfidf', TfidfTransformer()),
  ('dtree_clf',
   MultiOutputClassifier(estimator=DecisionTreeClassifier(), n_jobs=-1))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000020D70896310>),
 'tfidf': TfidfTransformer(),
 'dtree_clf': MultiOutputClassifier(estimator=DecisionTreeClassifier(), n_jobs=-1),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': 

In [18]:
parameters = {
    #'dtree_clf__estimator__criterion': ['gini', 'entropy'],
    'dtree_clf__estimator__min_samples_split': [2, 5],
    'dtree_clf__estimator__max_depth': [1, 2],
}

cv = GridSearchCV(pipeline, param_grid = parameters)

In [19]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

In [20]:
print(cv.best_params_)

{'dtree_clf__estimator__max_depth': 2, 'dtree_clf__estimator__min_samples_split': 5}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [21]:
#accuracy = (y_pred == y_test).mean()
#print(accuracy)

print(classification_report(y_test, y_pred, target_names = category_names, zero_division = 0))

                        precision    recall  f1-score   support

               related       0.76      1.00      0.86      5912
               request       0.60      0.48      0.53      1333
                 offer       0.00      0.00      0.00        30
           aid_related       0.72      0.30      0.43      3227
          medical_help       0.54      0.21      0.30       617
      medical_products       0.71      0.23      0.34       399
     search_and_rescue       0.67      0.22      0.34       223
              security       0.00      0.00      0.00       129
              military       0.43      0.07      0.12       275
           child_alone       0.00      0.00      0.00         0
                 water       0.89      0.40      0.55       503
                  food       0.75      0.73      0.74       860
               shelter       0.82      0.49      0.61       678
              clothing       0.73      0.56      0.63       111
                 money       0.44      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [22]:
# kNN classifier

pipeline_knn = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier(), n_jobs = -1)),
])

In [23]:
pipeline_knn.fit(X_train, y_train)

y_pred_knn = pipeline_knn.predict(X_test)

In [24]:
print(classification_report(y_test, y_pred_knn, target_names = category_names, zero_division = 0))

                        precision    recall  f1-score   support

               related       0.82      0.93      0.87      5912
               request       0.70      0.45      0.55      1333
                 offer       0.00      0.00      0.00        30
           aid_related       0.70      0.42      0.52      3227
          medical_help       0.55      0.08      0.14       617
      medical_products       0.65      0.08      0.13       399
     search_and_rescue       0.64      0.03      0.06       223
              security       0.00      0.00      0.00       129
              military       0.70      0.06      0.11       275
           child_alone       0.00      0.00      0.00         0
                 water       0.64      0.21      0.32       503
                  food       0.71      0.30      0.42       860
               shelter       0.66      0.19      0.29       678
              clothing       0.54      0.13      0.20       111
                 money       0.54      

In [25]:
# ADAboost classifier

pipeline_ada = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier())),
])

In [26]:
pipeline_ada.fit(X_train, y_train)

y_pred_ada = pipeline_ada.predict(X_test)

In [27]:
print(classification_report(y_test, y_pred_ada, target_names = category_names, zero_division = 0))

                        precision    recall  f1-score   support

               related       0.79      0.97      0.87      5912
               request       0.75      0.53      0.62      1333
                 offer       0.00      0.00      0.00        30
           aid_related       0.75      0.63      0.69      3227
          medical_help       0.62      0.29      0.39       617
      medical_products       0.65      0.32      0.43       399
     search_and_rescue       0.67      0.20      0.31       223
              security       0.17      0.05      0.07       129
              military       0.70      0.34      0.46       275
           child_alone       0.00      0.00      0.00         0
                 water       0.75      0.64      0.69       503
                  food       0.80      0.63      0.70       860
               shelter       0.77      0.56      0.65       678
              clothing       0.68      0.39      0.49       111
                 money       0.45      

In [39]:
pipeline_ada.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000020D70896310>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000020D70896310>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=AdaBoostClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf_

In [40]:
parameters = {
    'clf__estimator__n_estimators': [10, 50, 100],
    'clf__estimator__learning_rate': [0.01, 0.1, 1.0, 1.2],
}

cv = GridSearchCV(pipeline_ada, param_grid = parameters)

In [41]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

In [ ]:
print(cv.best_params_)

In [ ]:
print(classification_report(y_test, y_pred, target_names = category_names, zero_division = 0))

### 9. Export your model as a pickle file

In [ ]:
with open('model.pkl','wb') as f:
    pickle.dump(pipeline_ada,f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.